In [1]:
import sys
sys.path.insert(0, '..')
%load_ext autoreload
%autoreload 2

from libs.ssl_task import CPC
from libs.ssl_data import SSLHBNDataModule
import torch
import lightning as L

In [2]:
# load config from runs/config_CPC.yaml
import yaml
with open('../runs/config_CPC.yaml', 'r') as f:
    config = yaml.safe_load(f)

In [3]:
ssl_task = CPC()
config['data']['ssl_task'] = ssl_task
config['data']['window_len_s'] = 20
config['data']['num_workers'] = 2
litDataModule = SSLHBNDataModule(**config['data'])

In [ ]:
config['model']['init_args']['contextualizer_path'] = 'libs.ssl_model.BENDRContextualizer'
config['model']['init_args']['contextualizer_kwargs'] = {'in_features': 512, 'start_token': None}
config['model']['init_args']['learning_rate'] = 0.00005
cpc_model = CPCLit(downsampling_factor=96, **config['model']['init_args'])

config['trainer']['callbacks'] = None
config['trainer']['logger'] = None
config['trainer']['overfit_batches'] = 0.0
config['trainer']['fast_dev_run'] = False
config['trainer']['max_epochs'] = 1
config['trainer']['detect_anomaly'] = False # default: False
config['trainer']['gradient_clip_val'] = 0 # default: 0
trainer = L.Trainer(**config['trainer'])
trainer.fit(model=cpc_model, datamodule=litDataModule)

You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Using datasets: ['ds005505', 'ds005506', 'ds005507', 'ds005508', 'ds005509', 'ds005511', 'ds005512', 'ds005514', 'ds005515', 'ds005516']
Validation release: ds005505


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                | Params | Mode 
---------------------------------------------------------------
0 | encoder        | ConvEncoderBENDR    | 4.1 M  | train
1 | contextualizer | BENDRContextualizer | 153 M  | train
  | other params   | n/a                 | 512    | n/a  
---------------------------------------------------------------
157 M     Trainable params
0         Non-trainable params
157 M     Total params
629.230   Total estimated model params size (MB)
129       Modules in train mode
0         Modules in eval mode


learning rate 5e-05


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Number of datasets: 2567
Number of examples: 51234


Training: |          | 0/? [00:00<?, ?it/s]

train_loss tensor(7.6740, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss tensor(7.4982, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss tensor(3.7361, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss tensor(3.3492, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss tensor(2.6545, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss tensor(1.7920, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss tensor(1.5229, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss tensor(1.2385, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss tensor(0.9747, device='cuda:0', grad_fn=<NllLossBackward0>)



Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [10]:
import sys
sys.path.insert(0,'../')
from libs.ssl_task import CPC, Regression
from libs.ssl_data import SSLHBNDataModule
%load_ext autoreload
%autoreload 2

# load config from runs/config_CPC.yaml
import yaml
import lightning as L
with open('../runs/config_CPC.yaml', 'r') as f:
    config = yaml.safe_load(f)

ssl_task = CPC()
config['data']['ssl_task'] = ssl_task
config['data']['window_len_s'] = 20
config['data']['num_workers'] = 2

config['model']['init_args']['contextualizer_path'] = 'libs.bendr._BENDRContextualizer'
config['model']['init_args']['contextualizer_kwargs'] = {'in_features': 512, 'start_token': None} 
# config['model']['init_args']['contextualizer_path'] = 'libs.ssl_model.BENDRLSTM'
# config['model']['init_args']['contextualizer_kwargs'] = {'in_features': 512}
config['model']['init_args']['encoder_path'] = 'libs.bendr._ConvEncoderBENDR'
config['model']['init_args']['encoder_kwargs'] = {'encoder_h': 512, 'in_features': 128, 'dropout': 0.1} 

config['trainer']['callbacks'] = None
config['trainer']['logger'] = None

trainer = L.Trainer(**config['trainer'])
model = CPC.CPCLit(downsampling_factor=96, **config['model']['init_args'])
litDataModule = SSLHBNDataModule(**config['data'])
litDataModule.setup(stage='validate')

trainer.validate(model, litDataModule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
encoder _ConvEncoderBENDR(
  (encoder): Sequential(
    (Encoder_0): Sequential(
      (0): Conv1d(128, 512, kernel_size=(3,), stride=(3,), padding=(1,))
      (1): Dropout1d(p=0.1, inplace=False)
      (2): GroupNorm(256, 512, eps=1e-05, affine=True)
      (3): GELU(approximate='none')
    )
    (Encoder_1): Sequential(
      (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): Dropout1d(p=0.1, inplace=False)
      (2): GroupNorm(256, 512, eps=1e-05, affine=True)
      (3): GELU(approximate='none')
    )
    (Encoder_2): Sequential(
      (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): Dropout1d(p=0.1, inplace=False)
      (2): GroupNorm(256, 512, eps=1e-05, affine=True)
      (3): GELU(approximate='none')
    )
    (Encoder_3): Sequential(
      (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): Dropout1d(p=0.1, inpl

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

/home/dung/eeg-ssl/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                Validate metric                ┃                 DataLoader 0                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│              val_Regressor/NRMSE              │              3.2363928030397906               │
│               val_Regressor/R2                │              0.9972208587941325               │
│           val_Regressor/concordance           │              0.9986080347139015               │
│               val_Regressor/mae               │              0.03617943175034919              │
│               val_Regressor/mse               │             0.002109759051655819              │
│         val_Regressor/subject_iqr_iqr         │             0.015112310647964478              │
│        val_Regressor/subject_iqr_mean         │              0.04387109726667404              │
│       val_Regressor/subject_iqr_median        │             0.042439963668584824              │
│         val_Regressor/subject_iqr_std         │             0.012273340485990047              │
│         val_Regressor/subject_std_iqr         │             0.0072903987020254135             │
│        val_Regressor/subject_std_mean         │              0.03536618873476982              │
│       val_Regressor/subject_std_median        │             0.033703435212373734              │
│         val_Regressor/subject_std_std         │             0.009250370785593987              │
│     val_Regressor/subject_with_mean_NRMSE     │              -1.5486361122814225              │
│      val_Regressor/subject_with_mean_R2       │              0.9990167469017639               │
│  val_Regressor/subject_with_mean_concordance  │              0.9995100672132881               │
│      val_Regressor/subject_with_mean_mae      │              0.02585827815051516              │
│      val_Regressor/subject_with_mean_mse      │             0.0007711652579146796             │
│    val_Regressor/subject_with_median_NRMSE    │              -1.6058215425519924              │
│     val_Regressor/subject_with_median_R2      │              0.9989427903499796               │
│ val_Regressor/subject_with_median_concordance │              0.9994733983453061               │
│     val_Regressor/subject_with_median_mae     │              0.02616651007325028              │
│     val_Regressor/subject_with_median_mse     │             0.0008291693704199779             │
│                val_loss_epoch                 │              6.8955841064453125               │
└───────────────────────────────────────────────┴───────────────────────────────────────────────┘

[{'val_loss_epoch': 6.8955841064453125,
  'val_Regressor/R2': 0.9972208587941325,
  'val_Regressor/concordance': 0.9986080347139015,
  'val_Regressor/NRMSE': 3.2363928030397906,
  'val_Regressor/mse': 0.002109759051655819,
  'val_Regressor/mae': 0.03617943175034919,
  'val_Regressor/subject_with_mean_R2': 0.9990167469017639,
  'val_Regressor/subject_with_median_R2': 0.9989427903499796,
  'val_Regressor/subject_with_mean_concordance': 0.9995100672132881,
  'val_Regressor/subject_with_median_concordance': 0.9994733983453061,
  'val_Regressor/subject_with_mean_NRMSE': -1.5486361122814225,
  'val_Regressor/subject_with_median_NRMSE': -1.6058215425519924,
  'val_Regressor/subject_with_mean_mse': 0.0007711652579146796,
  'val_Regressor/subject_with_median_mse': 0.0008291693704199779,
  'val_Regressor/subject_with_mean_mae': 0.02585827815051516,
  'val_Regressor/subject_with_median_mae': 0.02616651007325028,
  'val_Regressor/subject_iqr_mean': 0.04387109726667404,
  'val_Regressor/subject_iqr

## Evaluate using linear least square

### P_factor

In [4]:
import sys
sys.path.insert(0, '..')
%load_ext autoreload
%autoreload 2
from libs.ssl_task import CPC
from libs.evaluation import train_projection_layer_for_eval
from libs.ssl_data import SSLHBNDataModule
import yaml

with open('../runs/config_CPC.yaml', 'r') as f:
    config = yaml.safe_load(f)
ssl_task = CPC()
config['data']['ssl_task'] = ssl_task
config['data']['target_label'] = 'p_factor'
litDataModule = SSLHBNDataModule(**config['data'], train_percent=0.6)
litDataModule.setup(stage='fit')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from sklearn import linear_model
regressor = linear_model.LinearRegression()
scores = []
print('No training. At initialization')
model = CPC.CPCLit(**config['model']['init_args'])
score = train_projection_layer_for_eval(model, train_dataloader=litDataModule.train_dataloader(), val_dataloader=litDataModule.val_dataloader(), regressor=regressor)

No training. At initialization
Number of datasets: 1537
Number of examples: 30614
Using regression model: LinearRegression()


100%|██████████| 240/240 [00:33<00:00,  7.14it/s]


Train score 0.02144770319810918


100%|██████████| 22/22 [00:06<00:00,  3.42it/s]


Validation scores:
	R2: -0.0143
	concordance: 0.0147
	mse: 0.7700
	mae: 0.7133


In [7]:
from sklearn import linear_model
regressor = linear_model.LinearRegression()
scores = []
for version in range(10):
    print(f'loading model version {version}')
    model = CPC.CPCLit.load_from_checkpoint(f'/home/dung/eeg-ssl/notebooks/artifacts/model-1t2pzp1g:v{version}/model.ckpt')
    score = train_projection_layer_for_eval(model, train_dataloader=litDataModule.train_dataloader(), val_dataloader=litDataModule.val_dataloader(), regressor=regressor)

loading model version 0
Number of datasets: 1537
Number of examples: 30614
Using regression model: LinearRegression()


100%|██████████| 240/240 [00:33<00:00,  7.18it/s]


Train score 0.022288773288630104


100%|██████████| 22/22 [00:06<00:00,  3.39it/s]


Validation scores:
	R2: -0.0100
	concordance: 0.0182
	mse: 0.7668
	mae: 0.7107
loading model version 1
Number of datasets: 1537
Number of examples: 30614
Using regression model: LinearRegression()


...

Validation scores:
	R2: -0.0069
	concordance: 0.0193
	mse: 0.7644
	mae: 0.7109
loading model version 9
Number of datasets: 1537
Number of examples: 30614
Using regression model: LinearRegression()


100%|██████████| 240/240 [00:33<00:00,  7.11it/s]


Train score 0.020429607161398744


100%|██████████| 22/22 [00:06<00:00,  3.39it/s]


Validation scores:
	R2: -0.0076
	concordance: 0.0185
	mse: 0.7649
	mae: 0.7114


### Age

In [8]:
import sys
sys.path.insert(0, '..')
%load_ext autoreload
%autoreload 2
from libs.ssl_task import CPC
from libs.evaluation import train_projection_layer_for_eval
from libs.ssl_data import SSLHBNDataModule
import yaml

with open('../runs/config_CPC.yaml', 'r') as f:
    config = yaml.safe_load(f)
ssl_task = CPC()
config['data']['ssl_task'] = ssl_task
config['data']['target_label'] = 'age'
litDataModule = SSLHBNDataModule(**config['data'], train_percent=0.6)
litDataModule.setup(stage='fit')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from sklearn import linear_model
regressor = linear_model.LinearRegression()
scores = []
print('No training. At initialization')
model = CPC.CPCLit(**config['model']['init_args'])
score = train_projection_layer_for_eval(model, train_dataloader=litDataModule.train_dataloader(), val_dataloader=litDataModule.val_dataloader(), regressor=regressor)

No training. At initialization
Number of datasets: 1558
Number of examples: 31213
Using regression model: LinearRegression()


100%|██████████| 244/244 [00:34<00:00,  6.97it/s]


Train score 0.05355197005947365


100%|██████████| 23/23 [00:06<00:00,  3.54it/s]


Validation scores:
	R2: 0.0236
	concordance: 0.0696
	mse: 12.4674
	mae: 2.9064


In [10]:
from sklearn import linear_model
regressor = linear_model.LinearRegression()
scores = []
for version in range(10):
    print(f'loading model version {version}')
    model = CPC.CPCLit.load_from_checkpoint(f'/home/dung/eeg-ssl/notebooks/artifacts/model-1t2pzp1g:v{version}/model.ckpt')
    score = train_projection_layer_for_eval(model, train_dataloader=litDataModule.train_dataloader(), val_dataloader=litDataModule.val_dataloader(), regressor=regressor)

loading model version 0
Number of datasets: 1558
Number of examples: 31213
Using regression model: LinearRegression()


100%|██████████| 244/244 [00:34<00:00,  7.17it/s]


Train score 0.05116070796076333


100%|██████████| 23/23 [00:06<00:00,  3.56it/s]


Validation scores:
	R2: 0.0229
	concordance: 0.0644
	mse: 12.4761
	mae: 2.8999
loading model version 1
Number of datasets: 1558
Number of examples: 31213
Using regression model: LinearRegression()


...

Validation scores:
	R2: 0.0234
	concordance: 0.0660
	mse: 12.4701
	mae: 2.8874
loading model version 9
Number of datasets: 1558
Number of examples: 31213
Using regression model: LinearRegression()


100%|██████████| 244/244 [00:34<00:00,  7.08it/s]


Train score 0.05133968077316298


100%|██████████| 23/23 [00:06<00:00,  3.51it/s]


Validation scores:
	R2: 0.0242
	concordance: 0.0666
	mse: 12.4599
	mae: 2.8859


### Gender regression

In [15]:
import sys
sys.path.insert(0, '..')
%load_ext autoreload
%autoreload 2
from libs.ssl_task import CPC
from libs.evaluation import train_projection_layer_for_eval
from libs.ssl_data import SSLHBNDataModule
import yaml

with open('../runs/config_CPC.yaml', 'r') as f:
    config = yaml.safe_load(f)
ssl_task = CPC()
config['data']['ssl_task'] = ssl_task
config['data']['target_label'] = 'sex'
config['data']['mapping'] = {'M': -1.0, 'F': 1.0}
litDataModule = SSLHBNDataModule(**config['data'], train_percent=0.6)
litDataModule.setup(stage='fit')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from sklearn import linear_model
regressor = linear_model.LinearRegression()
scores = []
print('No training. At initialization')
model = CPC.CPCLit(**config['model']['init_args'])
score = train_projection_layer_for_eval(model, train_dataloader=litDataModule.train_dataloader(), val_dataloader=litDataModule.val_dataloader(), regressor=regressor)

No training. At initialization
Number of datasets: 1558
Number of examples: 31064
Using regression model: LinearRegression()


100%|██████████| 243/243 [00:35<00:00,  6.88it/s]


Train score 0.04880549600782824


100%|██████████| 23/23 [00:06<00:00,  3.31it/s]


Validation scores:
	R2: -0.0020
	concordance: 0.0444
	mse: 0.8661
	mae: 0.8628


In [ ]:
from sklearn import linear_model
regressor = linear_model.LinearRegression()
scores = []
for version in range(10):
    print(f'loading model version {version}')
    model = CPC.CPCLit.load_from_checkpoint(f'/home/dung/eeg-ssl/notebooks/artifacts/model-1t2pzp1g:v{version}/model.ckpt')
    score = train_projection_layer_for_eval(model, train_dataloader=litDataModule.train_dataloader(), val_dataloader=litDataModule.val_dataloader(), regressor=regressor)

loading model version 0
Number of datasets: 1558
Number of examples: 31064
Using regression model: LinearRegression()


100%|██████████| 243/243 [00:34<00:00,  7.13it/s]


Train score 0.06064943956538571


100%|██████████| 23/23 [00:06<00:00,  3.48it/s]


Validation scores:
	R2: 0.0059
	concordance: 0.0635
	mse: 0.8592
	mae: 0.8569
loading model version 1
Number of datasets: 1558
Number of examples: 31064
Using regression model: LinearRegression()


...

100%|██████████| 243/243 [00:34<00:00,  7.01it/s]


Train score 0.054138009360608086


100%|██████████| 23/23 [00:07<00:00,  3.19it/s]


Validation scores:
	R2: 0.0023
	concordance: 0.0559
	mse: 0.8624
	mae: 0.8601


: 